In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter

# Vorbereitung
Alle Notebooks wurden unter Python mit der Version `3.10.1` erstellt. Im Folgenden können die Version und die Pakete überprüft und ggf. installiert werden.


In [ ]:
from platform import python_version
print("Required Python version:", "3.10.1")
print("Your Python version:\t", python_version())

# Setup commands
# conda create -n chess python=3.10.1 -c conda-forge
# conda activate chess
# pip install -r src/requirements.txt
# conda install jupyter -c conda-forge
# jupyter notebook

# Chess-AI

Dieses Notebook führt alle einzelnen Dateien und Klassen zusammen und implementiert den Spielablauf einer Partie.
Folgende Dateien werden in dieser Reihenfolge importiert:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.

Anschließend wird das Spiel durchgeführt, mit der Möglichkeit die Partie danach auf der Festplatte zu speichern.

In [ ]:
from AIBaseClass import ChessAI, State
from Exercise01AI import Exercise01AI
from HumanPlayer import HumanPlayer
from StockfishPlayer import StockfishPlayer

In [ ]:
from typing import Dict
TransitionMap = Dict[int, State]

## Speichern der Partie
Optional kann die Partie hier im PGN-Format gespeichert werden.

In [ ]:
def parse_player_name(player: ChessAI):
    """Parse the type of the current player and its name into a single string"""
    return f"{type(player).__name__} - {player.name}" 

In [ ]:
from datetime import datetime
import chess.pgn
import re
import os

def save_game(board: chess.Board, transitions: TransitionMap, player1: ChessAI, player2: ChessAI, rep: str, duration: float, seed: int) -> None:
    """Saves last game to 'games/YYYY-mm-dd_HH-MM-SS.pgn' (based on current time)"""
    game = chess.pgn.Game.from_board(board)

    # set pgn headers
    game.headers["Event"] = "Chess-AI game"
    game.headers["Site"] = os.getlogin() # log the current user
    game.headers["Date"] = datetime.now().strftime("%d.%m.%Y")
    game.headers["Round"] = rep
    game.headers["White"] = parse_player_name(player1)
    game.headers["Black"] = parse_player_name(player2)
    game.headers["Duration"] = str(duration)
    game.headers["Seed"] = str(seed)
       
    # add transitions as comments to pgn file
    move_count = 0
    for node in game.mainline():
        move_count += 1
        state = transitions.get(move_count)
        if state:
            node.comment = str(state)
    
    # write game to pgn file
    filename = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    with open(f"../games/{filename}.pgn", 'w') as gamefile:
        # add a linebreak after each turn
        gamefile.write(re.sub(r" ([1-9][0-9]*)\.", r"\n\1.", str(game)))
        # add a newline to the end of the file
        gamefile.write("\n")

## Spielablauf
Hier wird ein neues Spiel initialisiert und ein fester Seed definiert. Der statische Seed dient dazu, sämtliche Zufallsfunktionen reproduzierbar zu machen.



In [ ]:
import chess
from IPython.display import clear_output, display
from time import sleep
from typing import Tuple

def run_single_game(player1: ChessAI, player2: ChessAI) -> Tuple[chess.Board, TransitionMap]:
    """Start a new chess game"""
    # Create a new board
    board = chess.Board()
    players = [player1, player2]

    # All players start with opening game
    previous_state = {player.name: State.OPENING_GAME for player in players}
    transitions = { 1: State.OPENING_GAME, 2: State.OPENING_GAME }
    move_count = 0

    # Show board before first move (for human player)
    display(board)
    
    # Statistics variable
    statistics = ""

    # Play game
    while True:
        for player in players:
            # Make next move
            player.make_turn(board)
            move_count += 1
            
            # Print board
            clear_output(wait=True)
            display(board)
            print(f'Current state ({player.name}): {player.state}\n')
            print(statistics, end='')

            # Watch game progress slowly
            # sleep(0.1)

            # Log state transitions
            if previous_state[player.name] != player.state:
                line = f"{player.name}: Transitioned from {previous_state[player.name]} to {player.state} after {1337} seconds\n"
                statistics += line
                print(line, end='')

                # Update state
                previous_state[player.name] = player.state
                
                # Pause at transition
                # input("Press enter to continue")

            # Exit if game has finished
            if player.state == State.FINISHED:
                outcome = board.outcome()
                assert outcome, 'Outcome is None in State.FINISHED!'
                if outcome.winner == chess.WHITE:
                    print(f"\nWhite won the match due to {str(outcome.termination)} with result {outcome.result()}!")
                elif outcome.winner == chess.BLACK:
                    print(f"\nBlack won the match due to {str(outcome.termination)} with result {outcome.result()}!")
                else:
                    print(f"\nThe game is a remis due to {str(outcome.termination)} with result {outcome.result()}!")
                # State.FINISHED is only detected AFTER the finishing move
                transitions[move_count - 1] = player.state
                return board, transitions
            else:
                transitions[move_count] = player.state

In [ ]:
import time

def run_games(player1: ChessAI, player2: ChessAI):
    seed = int(os.environ.get("CHESS_AI_SEED", 3))
    ChessAI.random.seed(seed)
    repetitions = int(os.environ.get("CHESS_AI_REPETITIONS", 1))

    for rep in range(repetitions):
        t_start = time.time()
        board, transitions = run_single_game(player1, player2)
        t_end = time.time()
        rep_str = f"{rep + 1}/{repetitions}"
        save_game(board, transitions, player1, player2, rep_str, t_end - t_start, seed)

    print(f"Used seed {seed} with {repetitions} repetitions")

In [ ]:
# Create Players
player1 = Exercise01AI("Player 1") # White
player2 = StockfishPlayer("Stockfish") # Black
run_games(player1, player2)